## **Parte 2 — Encapsulamiento y @property**

## *0) Objetivos de aprendizaje*

- Entender **encapsulamiento en Python** y diferencias vs C++.
- Usar **convenciones** de acceso: público, “protegido” (`_`), “privado” (`__` con *name mangling*).
- Implementar **propiedades** con `@property` para **validar** y **controlar** atributos (estilo pythonic).
- Aplicarlo al proyecto **Biblioteca** con la entidad `Book`.

## *1) Público vs “protegido” vs “privado” en Python*

💡 Explicación
- **Público**: acceso normal; forma parte de la API de la clase.
- **“Protegido”** (`_atributo`): **convención** de “uso interno”. No hay bloqueo real (diferente a C++).
- **“Privado”** (`__atributo`): activa **name mangling** → el atributo se renombra internamente a `_<NombreClase>__atributo`. No es inviolable, pero evita choques de nombres y accesos accidentales (especialmente en **herencia múltiple**).

> En C++ el compilador impide acceder a protected/private.

> En Python confiamos en convenciones y en interfaces con @property, abc y Protocol.

In [4]:
from __future__ import annotations

class AccessDemo:
    def __init__(self) -> None:
        self.public = "soy público"
        self._protected = "soy 'protegido' por convención"
        self.__private = "soy privado con name mangling"

    def read_inside(self) -> tuple[str, str, str]:
        """Acceso interno a los tres atributos."""
        return (self.public, self._protected, self.__private)

obj = AccessDemo()

# Público: libre acceso
pu = obj.public  # ✅
print(pu) # soy público

# "Protegido": accesible pero NO recomendado (convención)
pro = obj._protected  # ⚠️
print(pro) # soy 'protegido' por convención

# Privado: falla el acceso directo
try:
    pri = obj.__private  # ❌ AttributeError
    print(pri)
except AttributeError as e:
    print("Acceso directo a __private falló:", e) # Acceso directo a __private falló: 'AccessDemo' object has no attribute '__private'


soy público
soy 'protegido' por convención
Acceso directo a __private falló: 'AccessDemo' object has no attribute '__private'


In [5]:
# Acceso interno (válido)
print("Acceso interno:", obj.read_inside()) # ('soy público', "soy 'protegido' por convención", 'soy privado con name mangling')

# Name mangling (hack consciente)
print("Name-mangling:", obj._AccessDemo__private)  # ⚠️ úsalo solo para depuración/pedagogía # Name-mangling: soy privado con name mangling

Acceso interno: ('soy público', "soy 'protegido' por convención", 'soy privado con name mangling')
Name-mangling: soy privado con name mangling


## *2) `@property`: getters/setters pythonic (sin romper la API)*

💡 Explicación
- En C++ usarías `getPrice()/setPrice()`.
- En Python expones un **atributo** (`obj.price`) pero con **control** usando `@property`.
- Ventajas:
    - **Validación** al escribir.
    - **API estable**: si más adelante cambias la lógica interna, el **código cliente no se rompe**.
- Patrón recomendado: guarda el dato real en `_atributo` y expón la propiedad `atributo`.

🧪 Código (aplicado a `Book.price`)

In [6]:
from typing import ClassVar

class Book:
    """
    Entidad de dominio: Libro.
    - API pública: title, author, price (propiedad), currency (de clase)
    - Internos: _price (convención de protegido), __internal_code (privado)
    """
    DEFAULT_CURRENCY: ClassVar[str] = "USD"

    def __init__(self, title: str, author: str, price: float) -> None:
        self.title: str = title                    # público
        self.author: str = author                  # público
        self._price: float = float(price)          # "protegido": respaldo real de la propiedad
        self.__internal_code: str = "B-" + title[:3].upper()  # privado: ejemplo de dato interno

    # --- Propiedad 'price' (lectura) ---
    @property
    def price(self) -> float:
        """Precio de venta del libro (nunca negativo)."""
        return self._price

    # --- Propiedad 'price' (escritura con validación) ---
    @price.setter
    def price(self, value: float) -> None:
        if value < 0:
            raise ValueError("El precio no puede ser negativo")
        self._price = float(value)

    # Método público para exponer algo derivado de lo privado
    def reveal_code(self) -> str:
        """Devuelve un identificador interno legible (sin exponer el atributo privado directamente)."""
        return self.__internal_code

In [8]:
# Uso básico
b1 = Book("Clean Code", "Robert C. Martin", 39.99)
print(b1.price)      # llama al getter # 39.99
b1.price = 49.99     # llama al setter (valida)
print(b1.price)      # 49.99

39.99
49.99


In [10]:
# Validación
try:
    b1.price = -1.0
except ValueError as e:
    print("Validación OK:", e) # Validación OK: El precio no puede ser negativo

# Acceso a 'privado' correcto: a través de método público
print("Internal:", b1.reveal_code()) # Internal: B-CLE

# Name-mangling (no recomendado en producción)
print("Name-mangled:", b1._Book__internal_code)

Validación OK: El precio no puede ser negativo
Internal: B-CLE
Name-mangled: B-CLE


## *3) “Protegido” que SÍ usarás: respaldo de propiedades*

💡 Explicación
- En clases de dominio, el patrón `@property` + `self._campo` es **muy común**.
- `self._campo` indica “no escribas aquí desde afuera; usa la **propiedad**”.
- Permite centralizar reglas (p. ej., normalizar a `float`, evitar negativos, redondear, etc.).

🧪 Código (pattern recomendado)

In [11]:
class NormalizedFloat:
    """
    Descriptor simple para normalizar a float.
    (Te adelanto el concepto; descriptores profundos llegarán más adelante.)
    """
    def __set_name__(self, owner, name):
        self.private_name = "_" + name

    def __get__(self, obj, objtype=None) -> float:
        return getattr(obj, self.private_name)

    def __set__(self, obj, value) -> None:
        val = float(value)
        setattr(obj, self.private_name, val)

class BookWithDescriptor:
    price = NormalizedFloat()  # usa descriptor para respaldar a _price

    def __init__(self, title: str, price: float) -> None:
        self.title = title
        self.price = price  # pasa por el descriptor → float

b = BookWithDescriptor("Refactoring", "45.50")
print(b.price, type(b.price))  # 45.5 <class 'float'>


45.5 <class 'float'>


## *4) “Privado” (__) y herencia: evitar choques de nombres*


💡 Explicación
- En **herencia múltiple**, dos clases podrían declarar el mismo nombre interno.
- `__atributo` ayuda a **evitar colisiones** porque Python renombra internamente con el **prefijo de la clase**.
- Úsalo cuando **realmente** necesites que un detalle sea interno y no “choque” en subclases.

🧪 Código (colisión evitada con __x)

In [ ]:
class Base:
    def __init__(self) -> None:
        self.__token = "base"   # se renombra a _Base__token

    def token(self) -> str:
        return self.__token

class Mixin:
    def __init__(self) -> None:
        self.__token = "mixin"  # se renombra a _Mixin__token
    def token(self) -> str:
        return self.__token

# class Child(Mixin, Base):  
class Child(Base, Mixin): # Desde Base.token(): base
    def __init__(self) -> None:
        Base.__init__(self)
        Mixin.__init__(self)

c = Child() # Child -> Mixin -> Base -> object : # Desde Base.token(): mixin
            # Child -> Base -> Mixin -> object : # Desde Base.token(): base

# Los dos __token coexisten sin choque:
print("Desde Base.token():", c.token())
print("Acceso name-mangled Base:", c._Base__token)
print("Acceso name-mangled Mixin:", c._Mixin__token)


Desde Base.token(): base
Acceso name-mangled Base: base
Acceso name-mangled Mixin: mixin


## 5) *Buenas prácticas y “gotchas”*

✅ Recomendado
- Exponer API limpia: **propiedades** para campos con reglas.
- `_atributo` como respaldo de la propiedad.
- `__atributo` para detalles **muy internos** o para **evitar colisiones** en herencia múltiple.
- Acompañar con **tests rápidos** (e.g., `assert`/`pytest`).

⚠️ Gotchas
- `__getattr__`/`__getattribute__` (temas avanzados) pueden romper acceso si se usan mal.
- Asignar directamente `obj._campo` **salta validaciones**: úsalo solo si entiendes el impacto (debug o inicialización controlada).
- `__private` **no es seguridad** criptográfica: es solo ofuscación a nivel de nombre.

## **6) Aplicación al proyecto: Book con price y stock**

💡 Explicación

Ampliamos `Book` con:
- `price` como **propiedad** (no negativo).
- `stock` como **propiedad** (entero ≥ 0).
- Un privado `__internal_code` (ejemplo de dato “muy interno”).
- Método de dominio: `add_stock()` con validación.

🧪 Código (versión de proyecto)

In [27]:
from __future__ import annotations
from typing import ClassVar

class Book:
    DEFAULT_CURRENCY: ClassVar[str] = "USD"

    def __init__(self, title: str, author: str, price: float, stock: int = 0) -> None:
        self.title: str = title
        self.author: str = author
        # Respaldo "protegido"
        self._price: float = 0.0
        self._stock: int = 0
        # Privado (name mangling) solo como ejemplo
        self.__internal_code: str = f"B-{hash((title, author)) & 0xffff:x}"
        # Asignación vía propiedades para validar desde el inicio
        self.price = price
        self.stock = stock

    # ---- price ----
    @property
    def price(self) -> float:
        """Precio no negativo."""
        return self._price

    @price.setter
    def price(self, value: float) -> None:
        val = float(value)
        if val < 0:
            raise ValueError("El precio no puede ser negativo")
        self._price = val

    # ---- stock ----
    @property
    def stock(self) -> int:
        """Unidades disponibles (entero >= 0)."""
        return self._stock

    @stock.setter
    def stock(self, value: int) -> None:
        if not isinstance(value, int):
            raise TypeError("El stock debe ser int")
        if value < 0:
            raise ValueError("El stock no puede ser negativo")
        self._stock = value

    # Operación de dominio
    def add_stock(self, units: int) -> None:
        """Agrega unidades al stock (>=1)."""
        if not isinstance(units, int):
            raise TypeError("units debe ser int")
        if units <= 0:
            raise ValueError("units debe ser positivo")
        self._stock += units

    # API controlada para el dato privado
    def reveal_code(self) -> str:
        return self.__internal_code


In [28]:
# Pruebas rápidas
book = Book("Clean Code", "Robert C. Martin", 39.99, stock=2)
print(book.price) # 39.99
# book.price = 49.5
assert book.price == 39.99
assert book.stock == 2
book.price = 49.5
book.add_stock(3)
assert book.stock == 5

39.99


In [22]:


# Validaciones
try:
    book.price = -1
except ValueError:
    print("Validación OK: precio negativo")
else:
    raise AssertionError("Debió fallar precio negativo")

try:
    book.stock = -3.9
except ValueError:
    print("Validación OK: stock negativo")
except TypeError:
    print("Validación OK: stock deberia ser int (TypeError)")
else:
    raise AssertionError("Debió fallar stock negativo")

try:
    book.add_stock(0)
except ValueError:
    print("Validación OK: add_stock con 0")
else:
    raise AssertionError("Debió fallar add_stock con 0")

print("\nOK Parte 2 (Book con price/stock y privado). Código interno:", book.reveal_code())


Validación OK: precio negativo
Validación OK: stock deberia ser int (TypeError)
Validación OK: add_stock con 0

OK Parte 2 (Book con price/stock y privado). Código interno: B-7160
